In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
print(tf.__version__)

2.4.1


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [28]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [5]:
X_train.shape

(60000, 28, 28)

In [6]:
X_test.shape

(10000, 28, 28)

In [7]:
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))
X_train.shape

(60000, 28, 28, 1)

In [8]:
X_test.shape

(10000, 28, 28, 1)

In [16]:
pd.DataFrame(y_train).value_counts()

0    6000
1    6000
2    6000
3    6000
4    6000
5    6000
6    6000
7    6000
8    6000
9    6000
dtype: int64

In [9]:
# hp is nothig but hyperparameter since we are using kerastuner it will automatically get initialized
# the function name is our choice but function structure is predefined 
# we can also customize the below function as our requirement
# to get more info refer to https://keras-team.github.io/keras-tuner/
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        # 'conv_1_filter' is the name of filter
        # 32 is minimum no of filter and 128 is maximum no of filters
        # here hp.Int is taken because it need to fetch all values continuously
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # kernel_size it will choose either 3 or 5
        # here hp.Choice is taken because it will choose 3 or 5
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
      
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
      
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # there are total 10 classes from 0 to 9
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
# build model is name of function created above which is used as the parameter
# objective is on which basis the optimization must take place
# max trails is no of trails
# in each and every trail it will give us one model with good accuracy
# directory parameter will create a 'output' folder in current working directory
# inside the 'output' directory 'MnistFashion' directory is created
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                          directory='output',
                          project_name="MnistFashion")

In [18]:
# initially we run for small epochs after we get good model with best params we run for many epochs
tuner_search.search(X_train, y_train, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 03m 07s]
val_accuracy: 0.7786666750907898

Best val_accuracy So Far: 0.8828333616256714
Total elapsed time: 00h 12m 22s
INFO:tensorflow:Oracle triggered exit


In [19]:
# num_models gives no of best models in 5 trails if it is set to 1 we need one model
model = tuner_search.get_best_models(num_models=1)

In [22]:
len(model)

1

In [23]:
type(model)

list

In [24]:
# now model is type of list lets give indexing and get first model
model = tuner_search.get_best_models(num_models=1)[0]

In [25]:
type(model)

tensorflow.python.keras.engine.sequential.Sequential

In [26]:
# now model is sequential type
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        51232     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                614448    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 667,834
Trainable params: 667,834
Non-trainable params: 0
_________________________________________________________________


In [27]:
# by providing initial_epoch it will train after that epoch. So our training starts from 4th epoch
# because our model is trained till 3 epochs
# now the below model is model out of 5 trails it will choose best model
model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 39s 22ms/step - loss: 0.2815 - accuracy: 0.8952 - val_loss: 0.3451 - val_accuracy: 0.8777
Epoch 5/10
1688/1688 [==============================] - 36s 21ms/step - loss: 0.2484 - accuracy: 0.9065 - val_loss: 0.3258 - val_accuracy: 0.8850
Epoch 6/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.2269 - accuracy: 0.9158 - val_loss: 0.3620 - val_accuracy: 0.8788
Epoch 7/10
1688/1688 [==============================] - 37s 22ms/step - loss: 0.1901 - accuracy: 0.9292 - val_loss: 0.3581 - val_accuracy: 0.8812
Epoch 8/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.1551 - accuracy: 0.9431 - val_loss: 0.3794 - val_accuracy: 0.8885
Epoch 9/10
1688/1688 [==============================] - 31s 18ms/step - loss: 0.1454 - accuracy: 0.9474 - val_loss: 0.4159 - val_accuracy: 0.8865
Epoch 10/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.1268 - accuracy: 0.9552 - val_loss: 0.4849 